## Softmax Classification 구현하기

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# unpack = True 로 가져오면, 텍스트 데이터에서 행 단위 데이터 규칙을 열로 바꿔서 가져오게 된다. (Transpose)
xy_true = np.loadtxt('train_softmax.txt', unpack=True, dtype='float32')
xy_true

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  3.,  3.,  5.,  7.,  2.,  6.,  7.],
       [ 1.,  2.,  4.,  5.,  5.,  5.,  6.,  7.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [3]:
# unpack=false 로 하면, 텍스트 데이터에서 행 단위 데이터 규칙을 그대로 들고 들어오게 된다.
xy_false = np.loadtxt('train_softmax.txt', unpack=False, dtype='float32')
xy_false

array([[ 1.,  2.,  1.,  0.,  0.,  1.],
       [ 1.,  3.,  2.,  0.,  0.,  1.],
       [ 1.,  3.,  4.,  0.,  0.,  1.],
       [ 1.,  5.,  5.,  0.,  1.,  0.],
       [ 1.,  7.,  5.,  0.,  1.,  0.],
       [ 1.,  2.,  5.,  0.,  1.,  0.],
       [ 1.,  6.,  6.,  1.,  0.,  0.],
       [ 1.,  7.,  7.,  1.,  0.,  0.]], dtype=float32)

In [4]:
# unpack = False 로 가지고 왔다면, 우리가 원하는 matmul 를 하기 위해서 다음과 같은 작업을 해주어야 함.
x_data1 = np.transpose(xy_false[:,0:3])
x_data1

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  3.,  3.,  5.,  7.,  2.,  6.,  7.],
       [ 1.,  2.,  4.,  5.,  5.,  5.,  6.,  7.]], dtype=float32)

In [5]:
# unpack = Ture 로 가지고 왔따면, 우리가 원하는 matmul 을 하기 위해서 다음과 같은 작업을 해주어야 함.
x_data2 = xy_true[0:3]
x_data2

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  3.,  3.,  5.,  7.,  2.,  6.,  7.],
       [ 1.,  2.,  4.,  5.,  5.,  5.,  6.,  7.]], dtype=float32)

In [6]:
y_data = xy_true[3:]
y_data

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [7]:
X = tf.placeholder("float", [3, None]) # x0 = 1, x1, x2
Y = tf.placeholder("float", [3, None]) # A, B ,C => 3 classes

In [8]:
W = tf.Variable(tf.zeros([3, 3]))

In [9]:
# define hypothesis 
hypo = tf.nn.softmax(tf.matmul(W,X))

In [10]:
# cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypo), reduction_indices=0))

In [11]:
alpha = 0.001
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(cost)

In [12]:
init = tf.initialize_all_variables()

In [13]:
sess = tf.Session()
sess.run(init)
for step in range(2000):
    sess.run(train, feed_dict={X:x_data2, Y:y_data})
    if step % 200 ==0 :
        print("step : ", step)
        print(sess.run(cost, feed_dict={X:x_data2, Y:y_data}))
        print(sess.run(W))
        print('='*50)

test_a = np.transpose([[1, 11, 7]])
a = sess.run(hypo, feed_dict={X:test_a})
print(a, sess.run(tf.arg_max(a,1)))

step :  0
2.07782
[[ 0.          0.00053125  0.00053125]
 [ 0.          0.00010938  0.00023438]
 [ 0.         -0.00064063 -0.00076563]]
step :  200
1.85745
[[  1.19209309e-10   9.07488465e-02   9.23793986e-02]
 [ -1.93715127e-10   1.51096852e-02   3.99405174e-02]
 [  8.19563445e-11  -9.95296910e-02  -1.24555431e-01]]
step :  400
1.75546
[[  2.53319671e-10   1.56199083e-01   1.62881523e-01]
 [ -6.25848817e-10   2.03543939e-02   6.92086965e-02]
 [ -2.71946299e-10  -1.59219965e-01  -2.05115169e-01]]
step :  600
1.70223
[[  1.41561096e-10   2.05197603e-01   2.19695553e-01]
 [ -5.51343193e-10   1.99079700e-02   9.19030607e-02]
 [  8.28877383e-11  -1.97819471e-01  -2.59295166e-01]]
step :  800
1.67124
[[  4.13507478e-10   2.43036360e-01   2.67285287e-01]
 [ -6.14672868e-10   1.62074920e-02   1.10297851e-01]
 [ -1.31316499e-10  -2.24294260e-01  -2.96930581e-01]]
step :  1000
1.65154
[[  4.99188912e-10   2.73000717e-01   3.08299124e-01]
 [ -8.38190184e-10   1.07222311e-02   1.25766039e-01]
 [ 

### 잘 되었는지 확인하기 위해서

In [14]:
sess.run(W)

array([[  2.42145679e-11,   3.58409137e-01,   4.56833124e-01],
       [ -8.45640724e-10,  -2.20868848e-02,   1.80225194e-01],
       [ -7.72066411e-10,  -2.88001567e-01,  -3.81102502e-01]], dtype=float32)

In [15]:
a = sess.run(hypo, feed_dict={X:test_a})
a

array([[ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [16]:
test_a = np.transpose([[1, 11, 7]])
test_a

array([[ 1],
       [11],
       [ 7]])

In [17]:
a = sess.run(hypo, feed_dict={X:test_a})
print(a, sess.run(tf.arg_max(a,1)))

[[ 1.]
 [ 1.]
 [ 1.]] [0 0 0]


In [18]:
test_b = np.transpose([[1, 3, 4]])
test_b

array([[1],
       [3],
       [4]])

In [19]:
b = sess.run(hypo, feed_dict={X:test_b})
print(b, sess.run(tf.arg_max(b,1)))

[[ 1.]
 [ 1.]
 [ 1.]] [0 0 0]


In [20]:
test_c = np.transpose([[1, 1, 0]])
test_c

array([[1],
       [1],
       [0]])

In [21]:
c = sess.run(hypo, feed_dict={X:test_c})
print(c, sess.run(tf.arg_max(c, 1)))

[[ 1.]
 [ 1.]
 [ 1.]] [0 0 0]


In [22]:
test_all = test_a
test_all[-1]

array([7])